# 数据质控


In [ ]:
# 确保src目录在Python路径中
import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

sys.path.append(os.path.abspath("../"))

# 导入模块
from src.data_utils import filter_anomalous_attributes, identify_attributes, parse_petrel_file, preprocess_features
from src.feature_selection import select_best_features

output_dir = "output"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)


# 设置中文字体
plt.rcParams["font.family"] = "SimHei"  # 黑体 SimHei 支持中文
plt.rcParams["axes.unicode_minus"] = False  # 正常显示负号

## 导入地震数据


In [ ]:
data_H6_2_attr = parse_petrel_file("../data/H6-2_attr")

## 导入井震数据


In [ ]:
file_H6_2_well = "../data/well_processed.xlsx"
data_H6_2_well = pd.read_excel(file_H6_2_well, sheet_name="Sheet1")

# 只选择层位（Surface）为 H6-2 的行，并丢弃砂厚为 NaN 的行
data_H6_2_well_selected = (
    data_H6_2_well[data_H6_2_well["Surface"] == "H6-2"]
    .query("Well != 'PH6' and Well != 'PH8' and Well != 'PH3' and Well != 'PH2'")
    .replace(-999, np.nan)  # 将-999替换为NaN（通常-999是缺失值的代码）
    .dropna(subset=["Thickness of facies(1: Fine sand)"])  # 丢弃砂厚为NaN的行
    .reset_index(drop=True)  # 重置索引
)

# 显示筛选后的前几行数据
data_H6_2_well_selected.head()

## 提取共同属性


In [ ]:
# 获取地震属性列表
seismic_attr, _ = identify_attributes("../data/H6-2_attr")

# 提取Excel的属性列表（从第8列开始的所有列）
well_seismic_attr = data_H6_2_well.columns[7:].tolist()

# 计算两个列表的交集
common_attributes = list(set(seismic_attr) & set(well_seismic_attr))

# 打印结果
print(f"地震属性数量: {len(seismic_attr)}")
print(f"Excel属性数量: {len(well_seismic_attr)}")
print(f"共同属性数量: {len(common_attributes)}")
print("\n共同属性列表:")
for attr in common_attributes:
    print(f"- {attr}")

## 根据井点分布，缩小工区范围

In [ ]:
# 根据井点分布缩小工区范围
# 获取井点数据的X、Y范围
well_x_min = data_H6_2_well_selected["X"].min()
well_x_max = data_H6_2_well_selected["X"].max()
well_y_min = data_H6_2_well_selected["Y"].min()
well_y_max = data_H6_2_well_selected["Y"].max()

# 打印井点区域范围
print(f"井点数据X轴范围: {well_x_min:.2f} 到 {well_x_max:.2f}")
print(f"井点数据Y轴范围: {well_y_min:.2f} 到 {well_y_max:.2f}")

# 可选：为了展示井点聚集区域，可以扩大一定比例
expansion_factor = 1.5  # 扩展50%
x_padding = (well_x_max - well_x_min) * (expansion_factor - 1) / 2
y_padding = (well_y_max - well_y_min) * (expansion_factor - 1) / 2

# 应用扩展后的范围
well_area_x_min = well_x_min - x_padding
well_area_x_max = well_x_max + x_padding
well_area_y_min = well_y_min - y_padding
well_area_y_max = well_y_max + y_padding

# 筛选出井点范围内的地震数据
data_H6_2_attr_filtered = data_H6_2_attr[
    (data_H6_2_attr["X"] >= well_area_x_min)
    & (data_H6_2_attr["X"] <= well_area_x_max)
    & (data_H6_2_attr["Y"] >= well_area_y_min)
    & (data_H6_2_attr["Y"] <= well_area_y_max)
].copy()

# 统计过滤前后的数据量
original_size = len(data_H6_2_attr)
filtered_size = len(data_H6_2_attr_filtered)
reduction_percent = (1 - filtered_size / original_size) * 100

print(f"原始地震数据点数: {original_size}")
print(f"缩小范围后的地震数据点数: {filtered_size}")
print(f"数据量减少了: {reduction_percent:.2f}%")

# 可视化原始数据与筛选后的数据分布
plt.figure(figsize=(15, 10))

# 绘制地震数据点（使用抽样以避免过多点导致图像渲染缓慢）
sample_ratio = min(1.0, 5000 / len(data_H6_2_attr))
seismic_sample = data_H6_2_attr.sample(frac=sample_ratio)
plt.scatter(seismic_sample["X"], seismic_sample["Y"], color="lightgray", alpha=0.3, s=10, label="原始地震数据(抽样)")

# 绘制筛选后的地震数据
filtered_sample_ratio = min(1.0, 3000 / len(data_H6_2_attr_filtered))
filtered_sample = data_H6_2_attr_filtered.sample(frac=filtered_sample_ratio)
plt.scatter(filtered_sample["X"], filtered_sample["Y"], color="blue", alpha=0.5, s=15, label="筛选后的地震数据(抽样)")

# 绘制井点位置
plt.scatter(data_H6_2_well_selected["X"], data_H6_2_well_selected["Y"], color="red", s=80, marker="^", label="井点位置")

# 绘制筛选边界框
plt.axvline(x=well_area_x_min, color="red", linestyle="--", alpha=0.8)
plt.axvline(x=well_area_x_max, color="red", linestyle="--", alpha=0.8)
plt.axhline(y=well_area_y_min, color="red", linestyle="--", alpha=0.8)
plt.axhline(y=well_area_y_max, color="red", linestyle="--", alpha=0.8)

# 添加标题和图例
plt.title("地震数据与井点分布", fontsize=16)
plt.xlabel("X坐标", fontsize=14)
plt.ylabel("Y坐标", fontsize=14)
plt.legend(loc="upper right")
plt.grid(True, linestyle="--", alpha=0.7)

# 保存图片
plt.savefig(os.path.join(output_dir, "seismic_well_distribution.png"), dpi=300, bbox_inches="tight")
plt.show()

## 生成统计摘要


In [ ]:
# 筛选出质量良好的属性
good_attributes, anomalous_attributes, attribute_stats = filter_anomalous_attributes(
    seismic_data=data_H6_2_attr_filtered,
    well_data=data_H6_2_well_selected,
    common_attributes=common_attributes,
    ratio_threshold=5.0,  # 均值比值阈值
    range_ratio_threshold=10.0,  # 数值范围比值阈值
    std_ratio_threshold=10.0,  # 标准差比值阈值
    output_dir=None,  # 输出图表目录
    verbose=True,  # 打印详细信息
)

print("\n筛选后保留的质量良好属性:")
for attr in good_attributes:
    print(f"- {attr}")

## 随机森林重要性和相关性分析

In [ ]:
# 使用随机森林评估特征重要性并移除冗余特征
selected_features = select_best_features(
    well_data=data_H6_2_well_selected,
    attribute_columns=good_attributes,
    target_column="Thickness of facies(1: Fine sand)",
    n_features=5,
    corr_threshold=0.85,
    output_dir=output_dir,
    verbose=True,
)

# 输出特征选择结果
print("\n基于随机森林重要性和相关性分析的最佳特征:")
for i, feature in enumerate(selected_features):
    print(f"{i + 1}. {feature}")

## 制作融合属性

In [ ]:
# 创建融合属性
# 步骤1: 计算所选属性与砂厚的Spearman相关性
print("======== 创建融合属性 ========")
target_column = "Thickness of facies(1: Fine sand)"
correlation_weights = {}
min_corr_threshold = 0.15  # 最小相关性阈值，低于此值的属性将被排除

# 检查每个选定属性在井点数据中的有效性
print("检查属性在井点数据中的有效性:")
for feature in selected_features:
    nan_count = data_H6_2_well_selected[feature].isna().sum()
    print(
        f"属性 '{feature}' 在井点数据中的NaN值数量: {nan_count}/{len(data_H6_2_well_selected)} ({nan_count / len(data_H6_2_well_selected) * 100:.1f}%)"
    )

# 筛选出所有选定属性都有有效值的井点
valid_wells = data_H6_2_well_selected.dropna(subset=selected_features)
print(f"\n所有属性都有有效值的井点数量: {len(valid_wells)} / {len(data_H6_2_well_selected)}")

# 计算相关性权重 (使用所有非NaN井点)
for feature in selected_features:
    # 确保在计算相关性前排除NaN值
    valid_data = data_H6_2_well_selected.dropna(subset=[feature, target_column])

    if len(valid_data) > 1:  # 需要至少两个数据点计算相关性
        corr = valid_data[feature].corr(valid_data[target_column], method="spearman")
        # 使用绝对值，因为负相关也是一种关系
        abs_corr = abs(corr)
        # 如果相关性低于阈值，则不考虑此属性
        if abs_corr >= min_corr_threshold:
            # 保存原始相关性符号，以便后续调整权重的正负
            correlation_weights[feature] = corr
            print(f"属性 '{feature}' 与砂厚的Spearman相关性: {corr:.4f}")
        else:
            print(f"属性 '{feature}' 与砂厚的相关性过低 ({corr:.4f})，不纳入融合")
    else:
        print(f"警告: 属性 '{feature}' 有效数据点不足，无法计算相关性")

# 如果没有任何有效属性满足相关性阈值，则使用所有属性且权重相等
if len(correlation_weights) == 0:
    print("\n警告: 没有属性满足相关性阈值，将使用所有属性且权重相等")
    for feature in selected_features:
        correlation_weights[feature] = 1.0
        print(f"属性 '{feature}' 使用默认权重: 1.0")

# 步骤2: 标准化所有选定属性
print("\n标准化属性:")

# 使用preprocess_features函数获取统计信息
_, feature_stats = preprocess_features(
    data=data_H6_2_well_selected,
    attribute_columns=list(correlation_weights.keys()),
    missing_values=[-999],
    verbose=True,
)

# 打印标准化统计信息，保持与原代码相同的输出格式
for feature, stats in feature_stats.items():
    print(f"属性 '{feature}' 均值: {stats['mean']:.4f}, 标准差: {stats['std']:.4f}")

# 补充全局数据统计，与原代码保持一致
for feature in correlation_weights.keys():
    if feature not in feature_stats:
        print(f"错误: 属性 '{feature}' 没有有效数据")
        # 使用全局属性的统计量(如有可能)
        if feature in data_H6_2_attr.columns:
            global_mean = data_H6_2_attr[feature].mean()
            global_std = data_H6_2_attr[feature].std() or 1.0
            feature_stats[feature] = {"mean": global_mean, "std": global_std}
            print(f"  使用全局数据: 均值 = {global_mean:.4f}, 标准差 = {global_std:.4f}")
        else:
            # 最后的后备方案
            feature_stats[feature] = {"mean": 0.0, "std": 1.0}
            print(f"  使用默认值: 均值 = 0.0, 标准差 = 1.0")


# 步骤3: 创建改进的融合属性函数，处理NaN值
def create_fused_attribute(data, features, weights, stats):
    """
    基于选定特征和权重创建融合属性，处理NaN值

    参数:
        data (DataFrame): 包含特征的数据框
        features (list): 特征列表
        weights (dict): 每个特征的权重
        stats (dict): 每个特征的标准化统计信息

    返回:
        Series: 融合属性
    """
    # 初始化融合属性和权重累加器
    fused_attr = np.zeros(len(data))
    weight_sums = np.zeros(len(data))

    # 对每个特征进行标准化并加权融合
    for feature in features:
        if feature in weights and feature in stats:
            # 创建掩码标记有效值
            valid_mask = np.isfinite(data[feature].values)

            # 跳过完全没有有效值的特征
            if not np.any(valid_mask):
                print(f"警告: 特征 '{feature}' 在计算融合属性时没有有效值")
                continue

            # 只处理有效值
            # 标准化
            normalized_feature = np.zeros(len(data))
            feature_data = data[feature].values
            normalized_feature[valid_mask] = (feature_data[valid_mask] - stats[feature]["mean"]) / stats[feature]["std"]

            # 获取权重
            weight = weights[feature]

            # 累加贡献
            fused_attr[valid_mask] += normalized_feature[valid_mask] * weight
            weight_sums[valid_mask] += abs(weight)

    # 归一化融合结果，只处理有权重累加的位置
    has_weights = weight_sums > 0
    fused_attr[~has_weights] = np.nan  # 设置没有有效特征的位置为NaN
    fused_attr[has_weights] /= weight_sums[has_weights]  # 归一化有效位置

    # 报告NaN的数量
    nan_count = np.sum(~has_weights)
    if nan_count > 0:
        print(f"融合属性中有 {nan_count} 个点 ({nan_count / len(data) * 100:.2f}%) 因缺少有效特征而为NaN")

    return fused_attr


# 步骤4: 在井点数据上创建并验证融合属性
fused_attribute_well = create_fused_attribute(
    data_H6_2_well_selected, correlation_weights.keys(), correlation_weights, feature_stats
)

# 检查融合属性与目标的相关性
valid_mask = np.isfinite(fused_attribute_well) & np.isfinite(data_H6_2_well_selected[target_column])
if np.sum(valid_mask) >= 2:
    # 确保将两个数组都转换为NumPy数组
    fused_values = np.array(fused_attribute_well[valid_mask])
    target_values = np.array(data_H6_2_well_selected[target_column].values[valid_mask])

    # 计算相关系数
    fused_target_corr = np.corrcoef(fused_values, target_values)[0, 1]
    print(f"\n融合属性与砂厚的相关性: {fused_target_corr:.4f}")
else:
    print("\n警告: 没有足够的有效数据点计算相关性")
    fused_target_corr = np.nan

# 步骤5: 在整个工区的地震数据上创建融合属性
print("\n在整个工区创建融合属性...")
fused_attribute_seismic = create_fused_attribute(
    data_H6_2_attr,  # 注意这里使用原始的大工区数据
    correlation_weights.keys(),
    correlation_weights,
    feature_stats,
)

# 添加融合属性到地震数据中
data_H6_2_attr["Fused_Attribute"] = fused_attribute_seismic

# 步骤5.5: 数据诊断
# print("\n数据诊断:")
# print(f"砂厚数据点数量: {len(data_H6_2_well_selected[target_column])}")
# print(f"融合属性数据点数量: {len(fused_attribute_well)}")
# print(f"砂厚数据范围: {data_H6_2_well_selected[target_column].min()} 到 {data_H6_2_well_selected[target_column].max()}")
# print(f"融合属性有效值范围: {np.nanmin(fused_attribute_well)} 到 {np.nanmax(fused_attribute_well)}")
# print(f"砂厚数据中NaN值数量: {data_H6_2_well_selected[target_column].isna().sum()}")
# print(f"融合属性数据中NaN值数量: {np.isnan(fused_attribute_well).sum()}")

# # 检查数据中的极端值
# percentiles = [0, 1, 5, 25, 50, 75, 95, 99, 100]
# print("\n砂厚数据百分位数:")
# for p in percentiles:
#     try:
#         val = np.percentile(data_H6_2_well_selected[target_column], p)
#         print(f"{p}%: {val:.4f}")
#     except:
#         print(f"{p}%: 计算失败")

# print("\n融合属性百分位数 (仅有效值):")
# valid_fused = fused_attribute_well[np.isfinite(fused_attribute_well)]
# if len(valid_fused) > 0:
#     for p in percentiles:
#         try:
#             val = np.percentile(valid_fused, p)
#             print(f"{p}%: {val:.4f}")
#         except:
#             print(f"{p}%: 计算失败")
# else:
#     print("没有有效的融合属性值，无法计算百分位数")

# # 尝试可视化数据分布
# plt.figure(figsize=(12, 5))

# plt.subplot(1, 2, 1)
# plt.hist(data_H6_2_well_selected[target_column], bins=20, alpha=0.7)
# plt.title("砂厚分布")
# plt.xlabel("砂厚")
# plt.grid(alpha=0.3)

# plt.subplot(1, 2, 2)
# if len(valid_fused) > 0:
#     plt.hist(valid_fused, bins=min(20, len(valid_fused)), alpha=0.7)
#     plt.title("融合属性分布 (仅有效值)")
# else:
#     plt.text(0.5, 0.5, "没有有效的融合属性值",
#             horizontalalignment='center', verticalalignment='center',
#             transform=plt.gca().transAxes)
#     plt.title("融合属性分布")
# plt.xlabel("融合属性值")
# plt.grid(alpha=0.3)

# plt.tight_layout()
# plt.show()

# 步骤6: 可视化整个工区的融合属性分布
print("可视化融合属性分布...")
plt.figure(figsize=(16, 12))

# 只使用有效的融合属性值
valid_seismic = data_H6_2_attr.dropna(subset=["Fused_Attribute"]).copy()
print(
    f"可视化使用 {len(valid_seismic)}/{len(data_H6_2_attr)} 个有效融合属性点 ({len(valid_seismic) / len(data_H6_2_attr) * 100:.2f}%)"
)

# 创建散点图，颜色代表融合属性值
scatter = plt.scatter(
    valid_seismic["X"],
    valid_seismic["Y"],
    c=valid_seismic["Fused_Attribute"],
    cmap="viridis",
    s=3,  # 点的大小
)

# 将井点按砂厚分为三类
low_sand = data_H6_2_well_selected[data_H6_2_well_selected[target_column] < 0.1]
medium_sand = data_H6_2_well_selected[
    (data_H6_2_well_selected[target_column] >= 0.1) & (data_H6_2_well_selected[target_column] <= 25)
]
high_sand = data_H6_2_well_selected[data_H6_2_well_selected[target_column] > 25]

# 标记不同类别的井点位置（选择在viridis色谱上对比鲜明的颜色）
plt.scatter(
    low_sand["X"],
    low_sand["Y"],
    color="#FF5733",  # 红橙色
    s=60,
    marker="^",
    label=f"井点(砂厚<0.1米): {len(low_sand)}个",
    edgecolors="white",
    linewidth=1.5,
    zorder=10,
)

plt.scatter(
    medium_sand["X"],
    medium_sand["Y"],
    color="#FFFF00",  # 黄色
    s=60,
    marker="^",
    label=f"井点(砂厚0.1-25米): {len(medium_sand)}个",
    edgecolors="white",
    linewidth=1.5,
    zorder=10,
)

plt.scatter(
    high_sand["X"],
    high_sand["Y"],
    color="#FF00FF",  # 品红色
    s=60,
    marker="^",
    label=f"井点(砂厚>25米): {len(high_sand)}个",
    edgecolors="white",
    linewidth=1.5,
    zorder=10,
)

# 添加颜色条
colorbar = plt.colorbar(scatter)
colorbar.set_label("融合属性值", fontsize=14)

# 添加标题和标签
plt.title("融合属性分布", fontsize=18)
plt.xlabel("X坐标", fontsize=14)
plt.ylabel("Y坐标", fontsize=14)
plt.grid(True, linestyle="--", alpha=0.5)
plt.legend(loc="upper right", fontsize=12)

# 保存图片
plt.savefig(os.path.join(output_dir, "fused_attribute_distribution.png"), dpi=300, bbox_inches="tight")
plt.show()

# 步骤7: 对比井点数据中的砂厚与融合属性的关系
plt.figure(figsize=(10, 8))

# 确保数据没有NaN或无穷大值
valid_indices = np.isfinite(data_H6_2_well_selected[target_column]) & np.isfinite(fused_attribute_well)
x_data = data_H6_2_well_selected[target_column][valid_indices]
y_data = fused_attribute_well[valid_indices]

# 检查并打印数据点数量
print(f"有效数据点数量: {len(x_data)}")

if len(x_data) > 0:
    plt.scatter(x_data, y_data, alpha=0.7, s=70, color="blue")

    # 添加趋势线，使用try-except捕获可能的错误
    try:
        # 如果数据点少于2个，无法拟合线性趋势
        if len(x_data) >= 2:
            # 使用更稳健的方法计算趋势线
            from scipy import stats

            slope, intercept, r_value, p_value, std_err = stats.linregress(x_data, y_data)

            # 创建均匀分布的x值来绘制趋势线
            x_trend = np.linspace(x_data.min(), x_data.max(), 100)
            plt.plot(
                x_trend,
                slope * x_trend + intercept,
                "r--",
                linewidth=2,
                label=f"趋势线: y = {slope:.4f}x + {intercept:.4f} (r={r_value:.4f})",
            )
            print(f"趋势线斜率: {slope:.4f}, 截距: {intercept:.4f}, r值: {r_value:.4f}")
        else:
            print("警告: 数据点数量不足，无法拟合趋势线")
    except Exception as e:
        print(f"拟合趋势线时出错: {str(e)}")
        # 可以选择使用更简单的方法来估计趋势
        if len(x_data) >= 2:
            # 使用最简单的两点法估计斜率
            x_min, x_max = x_data.min(), x_data.max()
            if x_max > x_min:  # 避免除以零
                y_at_min = y_data[x_data == x_min].mean()
                y_at_max = y_data[x_data == x_max].mean()
                slope_simple = (y_at_max - y_at_min) / (x_max - x_min)
                intercept_simple = y_at_min - slope_simple * x_min

                # 绘制简化的趋势线
                x_trend = np.linspace(x_min, x_max, 100)
                plt.plot(
                    x_trend,
                    slope_simple * x_trend + intercept_simple,
                    "g--",
                    linewidth=2,
                    label=f"简化趋势线: y = {slope_simple:.4f}x + {intercept_simple:.4f}",
                )
                print(f"使用简化方法估计趋势线 - 斜率: {slope_simple:.4f}, 截距: {intercept_simple:.4f}")

    plt.title(f"井点砂厚与融合属性的关系 (相关性: {fused_target_corr:.4f})", fontsize=16)
    plt.xlabel("砂厚 (Fine sand)", fontsize=14)
    plt.ylabel("融合属性值", fontsize=14)
    plt.grid(True, linestyle="--", alpha=0.7)
    plt.legend()
else:
    plt.text(
        0.5,
        0.5,
        "没有有效数据点可显示",
        horizontalalignment="center",
        verticalalignment="center",
        transform=plt.gca().transAxes,
        fontsize=16,
    )
    plt.title("井点砂厚与融合属性的关系", fontsize=16)

# 保存图片
plt.savefig(os.path.join(output_dir, "sandstone_thickness_vs_fused_attribute.png"), dpi=300, bbox_inches="tight")
plt.show()